In [16]:
import rosbag
import pandas as pd


In [22]:
import rosbag
import pandas as pd

def read_bag_to_aligned_df(bag_path, topics):
    bag = rosbag.Bag(bag_path)
    
    gps_fix_timestamps = []
    topic_data = {topic: [] for topic in topics}
    
    for topic, msg, t in bag.read_messages(topics=topics):
        timestamp = t.to_sec()
        
        if topic == '/gps/fix':
            gps_fix_timestamps.append(timestamp)
            topic_data['/gps/fix'].append({
                'timestamp': timestamp,
                'latitude': msg.latitude,
                'longitude': msg.longitude,
                'altitude': msg.altitude
            })
        else:
            topic_data[topic].append({
                'timestamp': timestamp,
                'msg': msg
            })
    
    bag.close()
    
    # Create a DataFrame from GPS data
    gps_df = pd.DataFrame(topic_data['/gps/fix'])
    
    # Initialize an empty DataFrame with GPS timestamps
    aligned_df = pd.DataFrame({'timestamp': gps_df['timestamp']})
    
    for topic in topics:
        if topic == '/gps/fix':
            continue
        
        aligned_topic_data = []
        
        for gps_time in gps_fix_timestamps:
            # Find the closest timestamp for the current topic
            closest_msg = min(topic_data[topic], key=lambda x: abs(x['timestamp'] - gps_time))
            time_diff = abs(closest_msg['timestamp'] - gps_time)
            
            # Assuming a reasonable threshold for synchronization
            if time_diff < 0.1:  # Adjust this threshold as needed
                aligned_topic_data.append(closest_msg['msg'])
            else:
                aligned_topic_data.append(None)  # No close match found
        
        # Create a DataFrame for the aligned topic data
        aligned_topic_df = pd.DataFrame([{
            'timestamp': gps_time,
            'topic': topic,
            'msg': aligned_msg
        } for gps_time, aligned_msg in zip(gps_fix_timestamps, aligned_topic_data)])
        
        aligned_df = pd.concat([aligned_df, aligned_topic_df], axis=1)
    
    return aligned_df



In [31]:
# Example usage:
bag_path = '/workspace/rosbag/rc/free_drive1.bag'
topics = [
    '/gps/fix',
    '/camera/imu',
    '/novatel/oem7/bestpos',
    '/base_board/controller_cmd',
    # Add more topics as needed
]

df = read_bag_to_aligned_df(bag_path, topics)

# Display the DataFrame
print(df)

         timestamp     timestamp        topic  \
0     1.724054e+09  1.724054e+09  /camera/imu   
1     1.724054e+09  1.724054e+09  /camera/imu   
2     1.724054e+09  1.724054e+09  /camera/imu   
3     1.724054e+09  1.724054e+09  /camera/imu   
4     1.724054e+09  1.724054e+09  /camera/imu   
...            ...           ...          ...   
1579  1.724055e+09  1.724055e+09  /camera/imu   
1580  1.724055e+09  1.724055e+09  /camera/imu   
1581  1.724055e+09  1.724055e+09  /camera/imu   
1582  1.724055e+09  1.724055e+09  /camera/imu   
1583  1.724055e+09  1.724055e+09  /camera/imu   

                                                    msg     timestamp  \
0                                                  None  1.724054e+09   
1                                                  None  1.724054e+09   
2                                                  None  1.724054e+09   
3                                                  None  1.724054e+09   
4                                             

In [32]:
# Save the DataFrame to a CSV file
df.to_csv('/workspace/Documents/DynamicsLearning/RC/data.csv', index=False)

# Display a confirmation message
print("Data saved to 'aligned_data.csv'.")

Data saved to 'aligned_data.csv'.


/workspace/Documents/DynamicsLearning/RC
